# ChatAnthropic

[Anthropic](https://www.anthropic.com/) is an AI safety and research company. They are the creator of Claude.

This will help you getting started with Anthropic [chat models](/docs/concepts/#chat-models). For detailed documentation of all `ChatAnthropic` features and configurations head to the [API reference](https://api.js.langchain.com/classes/langchain_anthropic.ChatAnthropic.html).

## Overview
### Integration details

| Class | Package | Local | Serializable | [PY support](https://python.langchain.com/docs/integrations/chat/anthropic/) | Package downloads | Package latest |
| :--- | :--- | :---: | :---: |  :---: | :---: | :---: |
| [ChatAnthropic](https://api.js.langchain.com/classes/langchain_anthropic.ChatAnthropic.html) | [@langchain/anthropic](https://api.js.langchain.com/modules/langchain_anthropic.html) | ❌ | ✅ | ✅ | ![NPM - Downloads](https://img.shields.io/npm/dm/@langchain/anthropic?style=flat-square&label=%20&) | ![NPM - Version](https://img.shields.io/npm/v/@langchain/anthropic?style=flat-square&label=%20&) |

### Model features
| [Tool calling](/docs/how_to/tool_calling) | [Structured output](/docs/how_to/structured_output/) | JSON mode | [Image input](/docs/how_to/multimodal_inputs/) | Audio input | Video input | [Token-level streaming](/docs/how_to/chat_streaming/) | [Token usage](/docs/how_to/chat_token_usage_tracking/) | [Logprobs](/docs/how_to/logprobs/) |
| :---: | :---: | :---: | :---: |  :---: | :---: | :---: | :---: | :---: |
| ✅ | ✅ | ❌ | ✅ | ❌ | ❌ | ✅ | ✅ | ❌ | 

## Setup

You'll need to sign up and obtain an [Anthropic API key](https://www.anthropic.com/), and install the `@langchain/anthropic` integration package.

### Credentials

Head to [Anthropic's website](https://www.anthropic.com/) to sign up to Anthropic and generate an API key. Once you've done this set the `ANTHROPIC_API_KEY` environment variable:

```bash
export ANTHROPIC_API_KEY="your-api-key"
```

If you want to get automated tracing of your model calls you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

```bash
# export LANGCHAIN_TRACING_V2="true"
# export LANGCHAIN_API_KEY="your-api-key"
```

### Installation

The LangChain `ChatAnthropic` integration lives in the `@langchain/anthropic` package:

```{=mdx}

import IntegrationInstallTooltip from "@mdx_components/integration_install_tooltip.mdx";
import Npm2Yarn from "@theme/Npm2Yarn";

<IntegrationInstallTooltip></IntegrationInstallTooltip>

<Npm2Yarn>
  @langchain/anthropic
</Npm2Yarn>

```

## Instantiation

Now we can instantiate our model object and generate chat completions:

In [2]:
import { ChatAnthropic } from "@langchain/anthropic" 

const llm = new ChatAnthropic({
    model: "claude-3-haiku-20240307",
    temperature: 0,
    maxTokens: undefined,
    maxRetries: 2,
    // other params...
});

## Invocation

In [3]:
const aiMsg = await llm.invoke([
    [
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ],
    ["human", "I love programming."],
])
aiMsg

AIMessage {
  "id": "msg_01M9yt3aSqKJKM1RnZF4f44Q",
  "content": "Voici la traduction en français :\n\nJ'adore la programmation.",
  "additional_kwargs": {
    "id": "msg_01M9yt3aSqKJKM1RnZF4f44Q",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-haiku-20240307",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 29,
      "output_tokens": 20
    }
  },
  "response_metadata": {
    "id": "msg_01M9yt3aSqKJKM1RnZF4f44Q",
    "model": "claude-3-haiku-20240307",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 29,
      "output_tokens": 20
    },
    "type": "message",
    "role": "assistant"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 29,
    "output_tokens": 20,
    "total_tokens": 49
  }
}


In [4]:
console.log(aiMsg.content)

Voici la traduction en français :

J'adore la programmation.


## Chaining

We can [chain](/docs/how_to/sequence/) our model with a prompt template like so:

In [5]:
import { ChatPromptTemplate } from "@langchain/core/prompts"

const prompt = ChatPromptTemplate.fromMessages(
    [
        [
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}.",
        ],
        ["human", "{input}"],
    ]
)

const chain = prompt.pipe(llm);
await chain.invoke(
    {
        input_language: "English",
        output_language: "German",
        input: "I love programming.",
    }
)

AIMessage {
  "id": "msg_012gUKUG65teaois31W3bfGF",
  "content": "Ich liebe das Programmieren.",
  "additional_kwargs": {
    "id": "msg_012gUKUG65teaois31W3bfGF",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-haiku-20240307",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 23,
      "output_tokens": 11
    }
  },
  "response_metadata": {
    "id": "msg_012gUKUG65teaois31W3bfGF",
    "model": "claude-3-haiku-20240307",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 23,
      "output_tokens": 11
    },
    "type": "message",
    "role": "assistant"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 23,
    "output_tokens": 11,
    "total_tokens": 34
  }
}


## Multimodal inputs

Claude-3 models support image multimodal inputs. The passed input must be a base64 encoded image with the
filetype as a prefix (e.g. `data:image/png;base64,{YOUR_BASE64_ENCODED_DATA}`).
Here's an example:

In [9]:
import fs from "fs/promises";

import { ChatAnthropic } from "@langchain/anthropic";
import { HumanMessage } from "@langchain/core/messages";

const imageData = await fs.readFile("../../../../../examples/hotdog.jpg");
const llm2 = new ChatAnthropic({
  model: "claude-3-sonnet-20240229",
});
const multimodalMessage = new HumanMessage({
  content: [
    {
      type: "text",
      text: "What's in this image?",
    },
    {
      type: "image_url",
      image_url: {
        url: `data:image/jpeg;base64,${imageData.toString("base64")}`,
      },
    },
  ],
});

await llm2.invoke([multimodalMessage]);

AIMessage {
  "id": "msg_01AuGpm6xbacTwoUFdNiCnzu",
  "content": "The image shows a hot dog. It consists of a cylindrical bread roll or bun that has been sliced lengthwise, revealing the bright red hot dog sausage filling inside. The hot dog sausage appears to be made from seasoned and smoked meat. This classic fast food item is a popular snack or meal, commonly enjoyed at sporting events, cookouts, and casual eateries.",
  "additional_kwargs": {
    "id": "msg_01AuGpm6xbacTwoUFdNiCnzu",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-sonnet-20240229",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 276,
      "output_tokens": 88
    }
  },
  "response_metadata": {
    "id": "msg_01AuGpm6xbacTwoUFdNiCnzu",
    "model": "claude-3-sonnet-20240229",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 276,
      "output_tokens": 88
    },
    "type": "message",
    "role": "assi

See [the official docs](https://docs.anthropic.com/claude/docs/vision#what-image-file-types-does-claude-support)
for a complete list of supported file types.

## Agents

Anthropic models can be used to power [LangGraph.js](https://langchain-ai.github.io/langgraphjs/) agents:

```{=mdx}
<Npm2Yarn>
  @langchain/langgraph
</Npm2Yarn>
```

In [5]:
import { z } from "zod";

import { ChatAnthropic } from "@langchain/anthropic";
import { tool } from "@langchain/core/tools";
import { createReactAgent } from "@langchain/langgraph/prebuilt";
import { HumanMessage } from "@langchain/core/messages";

const agentModel = new ChatAnthropic({
  model: "claude-3-sonnet-20240229",
  temperature: 0,
});

const currentWeatherTool = tool(async () => "28 °C", {
  name: "get_current_weather",
  description: "Get the current weather in a given location",
  schema: z.object({
    location: z.string().describe("The city and state, e.g. San Francisco, CA"),
  }),
});

const agent = createReactAgent({
  llm: agentModel,
  tools: [currentWeatherTool],
});

const agentResult = await agent.invoke({
  messages: [
    new HumanMessage("What's the weather like in SF?")
  ]
});

console.log(agentResult.messages[agentResult.messages.length - 1]);

AIMessage {
  "id": "msg_01Y4nz4gM4LB57WHc3viGgRp",
  "content": "The current weather in San Francisco, CA is 28°C.",
  "additional_kwargs": {
    "id": "msg_01Y4nz4gM4LB57WHc3viGgRp",
    "type": "message",
    "role": "assistant",
    "model": "claude-3-sonnet-20240229",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 364,
      "output_tokens": 18
    }
  },
  "response_metadata": {
    "id": "msg_01Y4nz4gM4LB57WHc3viGgRp",
    "model": "claude-3-sonnet-20240229",
    "stop_reason": "end_turn",
    "stop_sequence": null,
    "usage": {
      "input_tokens": 364,
      "output_tokens": 18
    },
    "type": "message",
    "role": "assistant"
  },
  "tool_calls": [],
  "invalid_tool_calls": [],
  "usage_metadata": {
    "input_tokens": 364,
    "output_tokens": 18,
    "total_tokens": 382
  }
}


## Custom headers

You can pass custom headers in your requests like this:

In [6]:
import { ChatAnthropic } from "@langchain/anthropic";

const llmWithCustomHeaders = new ChatAnthropic({
  model: "claude-3-sonnet-20240229",
  maxTokens: 1024,
  clientOptions: {
    defaultHeaders: {
      "X-Api-Key": process.env.ANTHROPIC_API_KEY,
    },
  },
});

await llmWithCustomHeaders.invoke("Why is the sky blue?");

AIMessage {
  "id": "msg_0131WctMyzmv4myNQeuAituP",
  "content": "The sky appears blue because of the way sunlight interacts with the Earth's atmosphere. Here's a more detailed explanation:\n\n- Sunlight is made up of different wavelengths of light, including all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, violet).\n\n- As sunlight passes through the atmosphere, it encounters tiny molecules of oxygen and nitrogen in the air.\n\n- These gas molecules are very effective at scattering and deflecting the shorter wavelengths of light, which correspond to the colors in the blue and violet range of the spectrum.\n\n- The longer wavelengths of light (reds, oranges) are able to travel more directly through the atmosphere without getting scattered as much.\n\n- The scattered blue wavelengths of light get deflected in all directions by the atmospheric gases. This blue light gets scattered about everywhere in the sky, making the sky appear blue from the ground.\n\

## Tools

The following example demonstrates [how to call tools](/docs/how_to/tool_calling) with Anthropic models:

In [9]:
import { ChatAnthropic } from "@langchain/anthropic";
import { ChatPromptTemplate } from "@langchain/core/prompts";
import { z } from "zod";

const calculatorSchema = z.object({
  operation: z
    .enum(["add", "subtract", "multiply", "divide"])
    .describe("The type of operation to execute."),
  number1: z.number().describe("The first number to operate on."),
  number2: z.number().describe("The second number to operate on."),
});

const calculatorTool = {
  name: "calculator",
  description: "A simple calculator tool",
  input_schema: zodToJsonSchema(calculatorSchema),
};

const toolCallingLlm = new ChatAnthropic({
  model: "claude-3-haiku-20240307",
}).bindTools([calculatorTool]);

const toolPrompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "You are a helpful assistant who always needs to use a calculator.",
  ],
  ["human", "{input}"],
]);

// Chain your prompt and model together
const toolCallChain = toolPrompt.pipe(toolCallingLlm);

const toolResponse = await toolCallChain.invoke({
  input: "What is 2 + 2?",
});

toolResponse.tool_calls;

[
  {
    name: 'calculator',
    args: { number1: 2, number2: 2, operation: 'add' },
    id: 'toolu_01Wjh2R15J945ErnGQ79UrpA',
    type: 'tool_call'
  }
]


### `withStructuredOutput`

If you just need to get structured output from the model, Anthropic models support the [`.withStructuredOutput()`](/docs/how_to/structured_output/#the-.withstructuredoutput-method) method:

In [12]:
import { z } from "zod";

const joke = z.object({
  setup: z.string().describe("The setup of the joke"),
  punchline: z.string().describe("The punchline to the joke"),
  rating: z.number().optional().describe("How funny the joke is, from 1 to 10"),
});

const modelForStructuredOutput = new ChatAnthropic({
  model: "claude-3-haiku-20240307",
})

const structuredLlm = modelForStructuredOutput.withStructuredOutput(joke);

await structuredLlm.invoke("Tell me a joke about cats");

{
  setup: "Why don't cats play poker in the jungle?",
  punchline: 'Too many cheetahs!',
  rating: 7
}


## API reference

For detailed documentation of all ChatAnthropic features and configurations head to the API reference: https://api.js.langchain.com/classes/langchain_anthropic.ChatAnthropic.html